In [1]:
import Ativo as at
import FileManager as fman
import pickle
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
import Parameters
import Simulator
import StatsGatherer
import OptimizerSimulator
import BootstrapSimulator

pd.set_option('display.max_rows', 167) # opcional, questão de visualização para debugar caso 1

ModuleNotFoundError: No module named 'FileManager'

In [ ]:
fm = fman.FileManager()

In [ ]:
with open('AtivoDiaList.pkl', 'rb') as filehandle:
    ativo_dia_list = pickle.load(filehandle)

In [ ]:
pars = Parameters.ParametersSimple()

In [ ]:
sg = StatsGatherer.StatsGatherer(pars)

In [ ]:
sim = Simulator.Simulator(fm,ativo_dia_list,pars,sg)

In [ ]:
bss = BootstrapSimulator.BootstrapSimulator(sg)

In [ ]:
os = OptimizerSimulator.OptimizerSimulator(sim, bss, sg)

In [ ]:
pars.setFilterParameters(gap_threshold=0.1, F_high_threshold=9)
# pars.setFilterParameters(prevol_threshold=15000000)
sim.runFiltering()

In [ ]:
pars.setSimParameters(start_money = 10000,allocation=0.1)
# pars.setAlgoParameters(short_after=0, exit_target=0.4, exit_stop=0.4)
# pars.setAlgoParameters(short_after=0.2)

#sim.runSimulation()

# sim.saveTrades('case1_trades.pkl')
sim.openTrades('case1_trades.pkl')

In [ ]:
dfd = sg.filtereddf # DataFrame of filtered ativo-dias
dft = sg.tradesdf # DataFrame of trades

##### Algumas análises da simulação

In [ ]:
sg.printSimResults()

In [ ]:
sg.plotHistMinsToTrade()

In [ ]:
sg.plotEquityCurve()

##### Análise de algumas estratificações

In [ ]:
dftr = pd.merge(dft,dfd,how='inner',on=['name','date'])
dftr = dftr[['name','date','gap','factorF','profit','cum_profit']]
dftr

In [ ]:
bins = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2]
gap = np.clip( dftr['gap'], bins[0], bins[-1] )
dftr.groupby(pd.cut(gap, bins)).agg(
    winrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0)/(sum(x>=0) + sum(x<0)) ),
    lossrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0)/(sum(x>=0) + sum(x<0)) ),
    mean=pd.NamedAgg(column='profit',aggfunc=lambda x: x.mean()),
    mean_of_pos=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x>=0].mean()),
    mean_of_neg=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x<0].mean()),
    count=pd.NamedAgg(column='profit',aggfunc='count'),
    positives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0) ),
    negatives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0) )
)

In [ ]:
dftr.groupby(dftr['date'].dt.to_period('M')).agg(
    winrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0)/(sum(x>=0) + sum(x<0)) ),
    lossrate=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0)/(sum(x>=0) + sum(x<0)) ),
    mean=pd.NamedAgg(column='profit',aggfunc=lambda x: x.mean()),
    mean_of_pos=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x>=0].mean()),
    mean_of_neg=pd.NamedAgg(column='profit',aggfunc=lambda x: x[x<0].mean()),
    count=pd.NamedAgg(column='profit',aggfunc='count'),
    positives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x>=0) ),
    negatives=pd.NamedAgg(column='profit',aggfunc=lambda x: sum(x<0) )
)

##### Rodando Bootstrap

In [ ]:
bss.runBootstrap()

In [ ]:
sg.printBootstrapResults()

##### Códigos para rodar combinações de parâmetros e otimizar.

In [ ]:
# oficial que gostaríamos de explorar
os.runSimulationGroup(
    prevol_threshold = [800000],
    exit_target=[0.1, 0.2, 0.3, 0.4], 
    exit_stop=[0.1, 0.2, 0.3, 0.4], 
    short_after=[0, 0.1, 0.2, 0.3],
    gap_threshold=[0, 0.1, 0.2],
    F_high_threshold = [1,2,3,4,5]
)

In [ ]:
# an.saveGroupResults('erich01.pkl')
os.openGroupResults('erich01.pkl')

In [ ]:
# rodar para debugar
os.runSimulationGroup(
    prevol_threshold = [15000000],
    exit_target=[0.2, 0.3], 
    exit_stop=[0.3], 
    short_after=[0, 0.1],
    gap_threshold=[0.1],
    F_high_threshold = [1]
)

In [ ]:
sg.groupResults

In [ ]:
# sg.saveGroupResults('teste1.pkl')
# sg.appendGroupResults('teste1.pkl')
# sg.openGroupResults('teste1.pkl')

##### Descrição simulações já rodadas

##### Testes temporários em geral